<a href="https://colab.research.google.com/github/danahassanein/chatbot-with-python/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import numpy as np
import tflearn
import tensorflow as tf
from tensorflow import keras 
import json
import random
import nltk
nltk.download('punkt') 
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# load data
with open('data.json') as file:
    data = json.load(file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
print(data["intents"])

[{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'how old is tim', 'what is your age', 'how old are you', 'age?'], 'responses': ['I am 18 years old!', '18 years young!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Tim.', "I'm Tim!", "I'm Tim aka Tech With Tim."], 'context_set': ''}, {'tag': 'shop', 'patterns': ['Id like to buy something', 'whats on the menu', 'what do you reccommend?', 'could i get something to eat'], 'responses': ['We sell chocolate chip cookies for $2!', 'Cookies are on the men

In [41]:
training_sentences = []
training_labels = []
labels = []
responses = []

# Loop through data
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [59]:
encode = LabelEncoder()
encode.fit(training_labels)
training_labels = encode.transform(training_labels)

In [60]:
vocab_size = 10000
embedding_dim = 16
max_len = 20
trunc_type = 'post'
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, truncating=trunc_type, maxlen=max_len)

classes = len(labels)

In [50]:
model = tf.keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(classes, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 16)            160000    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_7 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 102       
Total params: 160,646
Trainable params: 160,646
Non-trainable params: 0
_________________________________________________________________


In [53]:
training_labels_final = np.array(training_labels)

In [54]:
EPOCHS = 500
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(padded, training_labels_final, epochs=EPOCHS)

Train on 26 samples
Epoch 1/500
26/26 [==============================] - 0s 16ms/sample - loss: 1.7917 - acc: 0.1923
Epoch 2/500
26/26 [==============================] - 0s 288us/sample - loss: 1.7908 - acc: 0.1923
Epoch 3/500
26/26 [==============================] - 0s 292us/sample - loss: 1.7900 - acc: 0.1923
Epoch 4/500
26/26 [==============================] - 0s 269us/sample - loss: 1.7892 - acc: 0.1923
Epoch 5/500
26/26 [==============================] - 0s 257us/sample - loss: 1.7884 - acc: 0.1923
Epoch 6/500
26/26 [==============================] - 0s 314us/sample - loss: 1.7879 - acc: 0.1923
Epoch 7/500
26/26 [==============================] - 0s 342us/sample - loss: 1.7875 - acc: 0.1923
Epoch 8/500
26/26 [==============================] - 0s 337us/sample - loss: 1.7869 - acc: 0.1923
Epoch 9/500
26/26 [==============================] - 0s 305us/sample - loss: 1.7863 - acc: 0.1923
Epoch 10/500
26/26 [==============================] - 0s 286us/sample - loss: 1.7857 - acc: 0.1923


In [57]:
def check():
    print('start talking with bot, Enter quit to exit')
    while True:
        string = input('Enter: ')
        if string == 'quit': break
        result = model.predict(pad_sequences(tokenizer.texts_to_sequences([string]),
                                             truncating=trunc_type, maxlen=max_len))
        category = enc.inverse_transform([np.argmax(result)]) # labels[np.argmax(result)]
        for i in data['intents']:
            if i['tag']==category:
                print(np.random.choice(i['responses']))

check()

start talking with bot, Enter quit to exit
Enter: hello
Hi there, how can I help?
Enter: quit


In [61]:
tf.keras.models.save_model(model, "chatbot")